In [1]:
import os
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
import itertools as IT
import glob
import datetime as dt
import time
import sys

from dateutil.relativedelta import relativedelta

# from xgboost import plot_importance
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler


# #Common Model Algorithms
# from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# import xgboost

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',500)
pd.set_option('max_info_columns',500)
pd.set_option('display.float_format', lambda x : '%.3f' % x )

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold


from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD

# Use GridSearchCV to find the best parameters.
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.model_selection import cross_val_score



data=pd.read_csv("PJT002_train.csv",encoding="UTF-8-sig")

data.shape

print('NON FIRE', round(data['fr_yn'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('FIRE', round(data['fr_yn'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

data.isnull().sum()

# 오브젝트 먼저 전처리

data.iloc[:,2].dtype=='O'

object_list=[]
for col in data:
    if data[col].dtype =="object":
        object_list.append(col)

# 수치형 데이터 따로 정리

object_data=data[object_list[:]]

numeric_list=list(set(data.columns.tolist()) - set(object_list))
numeric_list=sorted(numeric_list)
numeric_data=data[numeric_list[:]]


# 화재 발생 시기 데이터 타입 변경
object_data['dt_of_fr']=object_data['dt_of_fr'].astype('datetime64[ns]')

### bldng_us(사용용도)

object_data["주거용YN"]=object_data.lnd_us_sttn_nm.apply(lambda x: 1 if x =="단독" else x)
object_data["주거용YN"]=object_data.주거용YN.apply(lambda x: 1 if x =="연립" else x)
object_data["주거용YN"]=object_data.주거용YN.apply(lambda x: 1 if x =="아파트" else x)
object_data["주거용YN"]=object_data.주거용YN.apply(lambda x: 1 if x =="다세대" else x)
object_data["주거용YN"]=object_data.주거용YN.apply(lambda x: 1 if x =="주거기타" else x)
object_data['주거용YN']=object_data.주거용YN.apply(lambda x: 1 if x==1 else 0)


object_data["공업용YN"]=object_data.lnd_us_sttn_nm.apply(lambda x: 1 if x =="공업용" else x)
object_data["공업용YN"]=object_data.공업용YN.apply(lambda x: 1 if x =="공업기타" else x)
object_data["공업용YN"]=object_data.공업용YN.apply(lambda x: 1 if x =="발전소" else x)
object_data["공업용YN"]=object_data.공업용YN.apply(lambda x: 1 if x =="유해.혐오시설" else x)
object_data["공업용YN"]=object_data.공업용YN.apply(lambda x: 1 if x =="공업나지" else x)
object_data['공업용YN']=object_data.공업용YN.apply(lambda x: 1 if x==1 else 0)


object_data["농업용YN"]=object_data.lnd_us_sttn_nm.apply(lambda x: 1 if x =="자연림" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="답" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="전" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="조림" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="답기타" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="과수원" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="토지임야" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="임야기타" else x)
object_data["농업용YN"]=object_data.농업용YN.apply(lambda x: 1 if x =="목장용지" else x)
object_data['농업용YN']=object_data.농업용YN.apply(lambda x: 1 if x==1 else 0)




# object_data.fr_yn=object_data.fr_yn.apply(lambda x: 1 if x=="Y" else 0)

# 사용 용도에 따라 주거시설/편의시설 여부로 더미화

object_data.bldng_us.unique()

object_data['주거시설YN']=object_data.bldng_us.apply(lambda x: 1 if x=="단독주택" else x)
object_data['주거시설YN']=object_data.주거시설YN.apply(lambda x: 1 if x=="공동주택" else x)
object_data['주거시설YN']=object_data.주거시설YN.apply(lambda x: 1 if x==1 else 0)

object_data['편의시설YN']=object_data.bldng_us.apply(lambda x: 1 if x=="제1종근린생활시설" else x)
object_data['편의시설YN']=object_data.편의시설YN.apply(lambda x: 1 if x=="제2종근린생활시설" else x)
object_data['편의시설YN']=object_data.편의시설YN.apply(lambda x: 1 if x=="문화및집회시설" else x)
object_data['편의시설YN']=object_data.편의시설YN.apply(lambda x: 1 if x==1 else 0)

object_data.groupby("bldng_us").size().sort_values(ascending=False)

# 6자리 혹은 nan 혹은 10자리임을 알 수 있음.
#  sns.distplot(object_data.dt_of_athrztn.astype(str).apply(len))

### dt_of_athrztn(건물승인일자)

# 건축 날짜의 데이터 타입이 2종류, 따라서 이 모양을 연도 형식으로 바꾸어줄 필요가 있음.

object_data.dt_of_athrztn=object_data.dt_of_athrztn.apply(lambda x: str(x)[0:4] if len(str(x))==10 else x)
object_data.dt_of_athrztn=object_data.dt_of_athrztn.apply(lambda x: str(x)[0:4] if len(str(x))==8 else x)
object_data.dt_of_athrztn=object_data.dt_of_athrztn.apply(lambda x: str(x)[0:4] if len(str(x))==6 else x)

dirty_date_idx=object_data.dt_of_athrztn[object_data.dt_of_athrztn.astype(str).apply(len)>4].index

object_data.dt_of_athrztn[dirty_date_idx[0]]="1980"
object_data.dt_of_athrztn[dirty_date_idx[1]]="1890"
object_data.dt_of_athrztn[dirty_date_idx[2]]="1890"
object_data.dt_of_athrztn[dirty_date_idx[3]]="1990"
object_data.dt_of_athrztn[dirty_date_idx[4]]="1990"
object_data.dt_of_athrztn[dirty_date_idx[5]]="1982"
object_data.dt_of_athrztn[dirty_date_idx[6]]="1978"
object_data.dt_of_athrztn[dirty_date_idx[7]]="1994"

#결측치가 아닌 연도 데이터를 시각화 하는 코드

#sns.distplot(object_data.dt_of_athrztn[~(object_data.dt_of_athrztn.isna)()].astype(int))

말도안돼_연도=object_data.dt_of_athrztn[~(object_data.dt_of_athrztn.isna)()].astype(int)[object_data.dt_of_athrztn[~(object_data.dt_of_athrztn.isna)()].astype(int)>2019].index

object_data.dt_of_athrztn[말도안돼_연도]="1997"

있는_연도=object_data.dt_of_athrztn[~(object_data.dt_of_athrztn.isna())].astype(int)

object_data.dt_of_athrztn.fillna("1988",inplace=True)

object_data.dt_of_athrztn=object_data.dt_of_athrztn.astype(int)

object_data["건물연령"]=object_data.dt_of_athrztn.apply(lambda x: 2019-x)

### rd_sd_nm(도로측면명)

object_data.rd_sd_nm.unique()

data.isnull().sum(axis=1)[data.isnull().sum(axis=1)<150].index

object_data['차량통행YN']=object_data.rd_sd_nm.apply(lambda x: 1 if x=="세로한면(가)" else x)
object_data['차량통행YN']=object_data.차량통행YN.apply(lambda x: 1 if x=="세로각지(가)" else x)
object_data['차량통행YN']=object_data.차량통행YN.apply(lambda x: 1 if x=="소로각지" else x)
object_data['차량통행YN']=object_data.차량통행YN.apply(lambda x: 1 if x=="중로한면" else x)
object_data['차량통행YN']=object_data.차량통행YN.apply(lambda x: 1 if x=="중로각지" else x)
object_data['차량통행YN']=object_data.차량통행YN.apply(lambda x: 1 if x=="소로한면" else x)
object_data['차량통행YN']=object_data.차량통행YN.apply(lambda x: 1 if x==1 else 0)

object_data['대로인접YN']=object_data.rd_sd_nm.apply(lambda x: 1 if x=="중로각지" else x)
object_data['대로인접YN']=object_data.대로인접YN.apply(lambda x: 1 if x=="중로한면" else x)
object_data['대로인접YN']=object_data.대로인접YN.apply(lambda x: 1 if x=="광대세각" else x)
object_data['대로인접YN']=object_data.대로인접YN.apply(lambda x: 1 if x=="광대소각" else x)
object_data['대로인접YN']=object_data.대로인접YN.apply(lambda x: 1 if x=="광대로한면" else x)
object_data['대로인접YN']=object_data.대로인접YN.apply(lambda x: 1 if x==1 else 0)

### bldng_archtctr(건물구조)

object_data['목재구조YN']=object_data.bldng_archtctr.apply(lambda x: 1 if x=="일반목구조" else x)
object_data['목재구조YN']=object_data.목재구조YN.apply(lambda x: 1 if x=="통나무구조" else x)
object_data['목재구조YN']=object_data.목재구조YN.apply(lambda x: 1 if x=="목구조" else x)
object_data['목재구조YN']=object_data.목재구조YN.apply(lambda x: 1 if x==1 else 0)

### bldng_us_clssfctn(건물용도분류명)

### rgnl_ar_nm(용도지역지구명)

object_data.rgnl_ar_nm.unique()

object_data.groupby("rgnl_ar_nm").size().sort_values(ascending=False)

object_data.bldng_us_clssfctn.unique()

object_data.groupby("bldng_us_clssfctn").size().sort_values(ascending=False)

object_data['주거용YN']=object_data.bldng_us_clssfctn.apply(lambda x: 1 if x=="주거용" else 0)

object_data['target_value']=object_data.fr_yn.apply(lambda x: 1 if x=="Y" else 0)



### emd_nm(행정구역명) 데이터

sns.distplot(object_data.emd_nm.astype(str).apply(len))

object_data.emd_nm[object_data.emd_nm.astype(str).apply(len)[object_data.emd_nm.astype(str).apply(len)<4].index]


object_data.emd_nm=object_data.emd_nm.str[4:]

object_data.emd_nm[object_data.emd_nm.notnull()]

object_data.emd_nm[object_data.emd_nm.isnull()]

object_data["시YN"]=object_data.emd_nm[object_data.emd_nm.notnull()].apply(lambda x: "1" if x[3]=="시" else "0")



object_data.groupby("emd_nm").size().sort_values(ascending=False).sample(100)

# 수치형 데이터

numeric_list_not_engry=[s for s in numeric_list if not "engry" in s]


numeric_data_on=numeric_data.loc[:,numeric_list_not_engry]

numeric_data_on.describe()

for col in numeric_data_on:
    try:
        if data[col].dtype !='object':
            sns.distplot(data[col],label=col)
            plt.show()
    except ValueError:
        pass
    
    

DF=object_data.join(numeric_data_on)

DF.columns

data=DF.drop(["dt_of_fr","fr_yn","bldng_us","bldng_archtctr","dt_of_athrztn","bldng_us_clssfctn","jmk","rgnl_ar_nm","rgnl_ar_nm2","lnd_us_sttn_nm","rd_sd_nm","emd_nm","mlt_us_yn",\
             "trgt_crtr","fr_fghtng_fclt_spcl_css_5_yn","fr_fghtng_fclt_spcl_css_6_yn","us_yn","dngrs_thng_yn","slf_fr_brgd_yn","blk_dngrs_thng_mnfctr_yn","cltrl_hrtg_yn"],axis=1)

data.fillna(0,inplace=True)



data_index=data["id"]

data=data.set_index("id")

# 스케일링 후 다시 칼럼 네임 붙이기
colnames=data.columns

transformer = RobustScaler().fit(data)
data=transformer.transform(data)


data=pd.DataFrame(data)

data.columns=colnames

print('NON FIRE', round(data['target_value'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('FIRE', round(data['target_value'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

X = data.drop('target_value', axis=1)
y = data['target_value']

# Feature Selection

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

feat_importances.sort_values(ascending=False)

중요_변수_리스트=feat_importances[feat_importances>0.002].index.tolist()

중요_변수_리스트.append("target_value")

data[중요_변수_리스트]

print('NON FIRE', round(data['target_value'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('FIRE', round(data['target_value'].value_counts()[1]/len(data) * 100,2), '% of the dataset')

X = data.drop('target_value', axis=1)
y = data['target_value']

# 타겟의 샘플 수
target_sample=data.groupby('target_value').count().iloc[1,1]
print(target_sample,"and",data.groupby('target_value').count().iloc[0,1])

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

# Random Under-Sampling

data = data.sample(frac=1)

# amount of fraud classes 492 rows.
Return = data.loc[data['target_value'] == 1]
Non_Return = data.loc[data['target_value'] == 0][:target_sample]

normal_distributed_df = pd.concat([Return, Non_Return])

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

new_df.head()

print('Distribution of the Classes in the subsample dataset')
print(new_df['target_value'].value_counts()/len(new_df))

colors = ["#0101DF", "#DF0101"]

sns.countplot('target_value', data=new_df, palette=colors)
plt.title('Equally Distributed Classes', fontsize=14)
plt.show()

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(24,20))

# Entire DataFrame
corr = data.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)
ax1.set_title("Imbalanced Correlation Matrix \n (don't use for reference)", fontsize=14)


sub_sample_corr = new_df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2)
ax2.set_title('SubSample Correlation Matrix \n (use for reference)', fontsize=14)
plt.show()

# New_df is from the random undersample data (fewer instances)
X_us = new_df.drop('target_value', axis=1)
y_us = new_df['target_value']

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(X_us, y_us, test_size=0.2, random_state=42)

# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# Let's implement simple classifiers

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

# Wow our scores are getting even high scores even when applying cross validation.

for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 4) * 100, "% accuracy score")

best_xgb_model = xgboost.XGBClassifier(colsample_bytree=0.4,
                 gamma=0.3,                 
                 learning_rate=0.1,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=1e-05,
                 reg_lambda=0.01,
                 subsample=0.95,
                 )

best_xgb_model.fit(X_train,y_train)
training_score = cross_val_score(best_xgb_model, X_train, y_train, cv=5)

# 가장 우수한 성능을 보이는 SVC

model_SVC=SVC()
model_SVC.fit(X=X_train, y=y_train)

y_pred_SVC = model_SVC.predict(X_test)

# 선형 커널 만들어보기
from sklearn.svm import LinearSVC
model_LSVC = LinearSVC()
model_LSVC.fit(X_train,y_train)

# proba = model_LSVC.decision_function(X_train)
# prob = (proba - proba.min()) / (proba.max() - proba.min())
# print('Prob[0]: %.3f' % (1-prob[0]))
# print('Prob[1]: %.3f' % (prob[0]))

cross_val_score(model_LSVC, X_train, y_train, cv=5)

# Logistic Regression model

model_LR =LogisticRegression()
model_LR.fit(X_train, y_train)


y_pred_LR = model_LR.predict(X_test)


model_LR.coef_

model_RF =RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
model_RF.fit(X_train, y_train)
# Random Forest model

y_pred_RF = model_RF.predict(X_test)

# for name, importance in zip(X_train, model_RF.feature_importances_):
#     print(name, "=", importance)

X_train1=pd.DataFrame(X_train)

# RF 모델의 피처 중요도 플롯

feat_importances = pd.Series(model_RF.feature_importances_, index=X_train1.columns)
feat_importances.nlargest(15).plot(kind='barh')

# y_pred = best_xgb_model.predict(X_test)

y_pred2 = best_xgb_model.predict(X.values)

y_pred2 = model_LSVC.predict(X)

#original_Xtest, original_ytest

예측=model_SVC.predict(original_Xtest)
정답=original_ytest

#original_Xtest, original_ytest

예측=model_SVC.predict(X_test)
정답=y_test

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(정답, 예측)
print(cm)

plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Versicolor or Not Versicolor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

import sklearn.metrics

print(sklearn.metrics.recall_score(정답,예측))
print(sklearn.metrics.precision_score(정답,예측))

print(sklearn.metrics.accuracy_score(정답,예측))
print("이게중요!",sklearn.metrics.f1_score(정답,예측))
print(sklearn.metrics.precision_recall_curve(정답,예측))

# Feature importance

In [ ]:
from matplotlib import pyplot as plt
from sklearn import svm

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

f_importances(model_SVC.coef_[0], list(X.columns))

In [ ]:
import imblearn

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV


print('Length of X (train): {} | Length of y (train): {}'.format(len(original_Xtrain), len(original_ytrain)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(original_Xtest), len(original_ytest)))

# List to append the score and then find the average
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []

# Classifier with optimal parameters
# log_reg_sm = grid_log_reg.best_estimator_
log_reg_sm = LogisticRegression()

rand_log_reg = RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)

In [ ]:
# Implementing SMOTE Technique 
# Cross Validating the right way
# Parameters
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
for train, test in sss.split(original_Xtrain, original_ytrain):
    pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    model = pipeline.fit(original_Xtrain[train], original_ytrain[train])
    best_est = rand_log_reg.best_estimator_
    prediction = best_est.predict(original_Xtrain[test])
    
    accuracy_lst.append(pipeline.score(original_Xtrain[test], original_ytrain[test]))
    precision_lst.append(precision_score(original_ytrain[test], prediction))
    recall_lst.append(recall_score(original_ytrain[test], prediction))
    f1_lst.append(f1_score(original_ytrain[test], prediction))
    auc_lst.append(roc_auc_score(original_ytrain[test], prediction))
    
print('---' * 45)
print('')
print("accuracy: {}".format(np.mean(accuracy_lst)))
print("precision: {}".format(np.mean(precision_lst)))
print("recall: {}".format(np.mean(recall_lst)))
print("f1: {}".format(np.mean(f1_lst)))
print('---' * 45)

In [ ]:
labels = ['No Fraud', 'Fraud']
smote_prediction = best_est.predict(original_Xtest)
print(classification_report(original_ytest, smote_prediction, target_names=labels))

n_classes=len(y_score)

# Light GBM 모델

from matplotlib import pyplot as plt
from sklearn import svm

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

f_importances(model_SVC.coef_[0], list(X.columns))
import imblearn

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV


print('Length of X (train): {} | Length of y (train): {}'.format(len(original_Xtrain), len(original_ytrain)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(original_Xtest), len(original_ytest)))

# List to append the score and then find the average
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []

# Classifier with optimal parameters
# log_reg_sm = grid_log_reg.best_estimator_
log_reg_sm = LogisticRegression()




rand_log_reg = RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)

In [ ]:
test=pd.read_csv("PJT002_test.csv")



object_list=[]
for col in test:
    if test[col].dtype =="object":
        object_list.append(col)

object_list=[]
for i in range(len(data.columns.tolist())):
    if data[data.columns.tolist()[i]].dtype == 'O':
        object_list.append(data.columns.tolist()[i])


object_data=test[object_list[:]]

numeric_list=list(set(test.columns.tolist()) - set(object_list))
numeric_list=sorted(numeric_list)
numeric_data=test[numeric_list[:]]



object_data['dt_of_fr']=object_data['dt_of_fr'].astype('datetime64[ns]')

# object_data.fr_yn=object_data.fr_yn.apply(lambda x: 1 if x=="Y" else 0)


object_data['주거시설YN']=object_data.bldng_us.apply(lambda x: 1 if x=="단독주택" else x)
object_data['주거시설YN']=object_data.주거시설YN.apply(lambda x: 1 if x=="공동주택" else x)
object_data['주거시설YN']=object_data.주거시설YN.apply(lambda x: 1 if x==1 else 0)

object_data['편의시설YN']=object_data.bldng_us.apply(lambda x: 1 if x=="제1종근린생활시설" else x)

object_data['편의시설YN']=object_data.편의시설YN.apply(lambda x: 1 if x=="제2종근린생활시설" else x)

object_data['편의시설YN']=object_data.편의시설YN.apply(lambda x: 1 if x=="문화및집회시설" else x)

object_data['편의시설YN']=object_data.편의시설YN.apply(lambda x: 1 if x==1 else 0)
​

numeric_data_on=numeric_data.iloc[:,[0,1,2,3,4,5,6,68,69,70,71,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158]]